# Merged_Global_Pluvial_Floods_DataBase

## Download the Input Data from Zenodo

In [1]:
%%bash 

# ------------------------ #
# INPUT VARIABLES
DOI="10.5281/zenodo.3633750" 
# ------------------------ #

zenodo_get.py $DOI
unzip Data.zip
rm -f Data.zip md5sums.txt

Archive:  Data.zip
   creating: Data/
   creating: Data/CPC_GLOBAL_PRCP_V1.0/
  inflating: Data/CPC_GLOBAL_PRCP_V1.0/precip.2016.nc  
  inflating: Data/CPC_GLOBAL_PRCP_V1.0/precip.2017.nc  
  inflating: Data/LICENSE.md         
   creating: Data/RawDBs/
   creating: Data/RawDBs/ESWD/
  inflating: Data/RawDBs/ESWD/ESWD.csv  
   creating: Data/RawDBs/EMDAT/
  inflating: Data/RawDBs/EMDAT/Fatima Pillosu_2019-10-23.csv  
   creating: Data/RawDBs/FL/
  inflating: Data/RawDBs/FL/floodlist_events_yr2016.csv  
  inflating: Data/RawDBs/FL/floodlist_events_yr2017.csv  
   creating: Data/RawDBs/SED/
  inflating: Data/RawDBs/SED/StormEvents_details-ftp_v1.0_d2017_c20200121.csv  
  inflating: Data/RawDBs/SED/StormEvents_details-ftp_v1.0_d2016_c20190817.csv  
  inflating: Data/README.md          
  inflating: Data/.Rhistory          
  inflating: Data/DataTree.png       


Title: Merged Global Pluvial Floods DataBase (Input Data)
Keywords: Flood Reports, FloodList, Emergency Events Database, European Severe Weather Database, Storm Events Database
Publication date: 2020-02-01
DOI: 10.5281/zenodo.3633750
Total size: 130.4 MB

Link: https://zenodo.org/api/files/d8da7cd7-961e-43d3-901a-51b4f5a78c64/Data.zip   size: 130.4 MB

Checksum is correct. (2c1014bf741677086f8ab5d966d47cde)
All files have been downloaded.
